In [ ]:
# Imports
import os
import time

import csv
import re
import numpy as np
import pandas as pd
# import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import FeatureUnion

In [ ]:
# dir
translated_csv_dir = r"C:\Users\Asher\Documents\School\_Scriptie\Data\Data_Userlines_CSV"
ngram_csv_dir = r"C:\Users\Asher\Documents\School\_Scriptie\Data\Data_Ngram"

In [ ]:
# return list met dict [{Land1}, {Land 1}]
def get_csv_files(country_selection):
    os.chdir(translated_csv_dir)
    
    country_return_list = []

    # loop door alle folders die hierboven zijn geprint
    for country in os.listdir():
        
        # filter op specifiek land (IN BOX 2)
        if country in country_selection:
            paths_dict = {}

            # ga door alle inhoud van de landfolder heen
            for root, dirs, files in os.walk(country):
                file_data = []
                
                # loop door files van een folder
                for file in files:
                    
                    file_data.append(file)

                paths_dict[root] = file_data

            country_return_list.append(paths_dict)
        
    return country_return_list

In [ ]:
# haal jaar maand en dag uit filename
def extract_file_date(file_name):
    
    year_month_day = re.search(r"\d{4}-\d{2}-\d{2}", file_name)
#     year, month, day = year_month_day[0].split("-")
    
    return year_month_day[0]

### Shingles uit CSV

In [ ]:
# geeft woorden en hoevaak deze voorkomen
def get_victor_from_csv(path):
    os.chdir(translated_csv_dir)
    
    df = pd.read_csv(path)
    
    lines = df["value"].to_list()
    
    victor = CountVectorizer(ngram_range=(1, 5))
    
    print(victor.fit_transform(lines))
    counts = victor.fit_transform(lines).toarray()
    headings = victor.get_feature_names_out()
    
    return headings, counts

In [ ]:
# functie testruimte
# df_for_the_test = os.path.join("NL", "ParlaMint-NL_2014-04-16-tweedekamer-2.csv")
df_for_the_test = os.path.join("NL", "ParlaMint-NL_2014-12-18-tweedekamer-25.csv")

victor_output_headings, victor_output_counts = get_victor_from_csv(df_for_the_test)
# print(len(victor_output_headings))
# print(type(victor_output_counts))
count_vect_df = pd.DataFrame(victor_output_counts, columns=victor_output_headings)

display(count_vect_df)

In [ ]:
# NOTE: Te langzame functie
# geeft een merged vectorizer terug
# def get_merged_victor(names1, vals1, names2, vals2):
#     names = np.unique(np.append(names1, names2))
#     vals = np.zeros(shape=(1,len(names)), dtype=int)
    
#     # voegt waarden toe aan vals
#     def merge_vals(names_to_merge, vals_to_merge):
#         m_vals_to_merge = np.zeros(shape=(1,len(vals_to_merge[0])), dtype=int)
        
#         # merge de rijen tot een enkele rij
#         if len(vals_to_merge) != 1:
            
#             for row in vals_to_merge:
#                 m_vals_to_merge = m_vals_to_merge + row
                
#         else:
            
#             m_vals_to_merge = vals_to_merge
        
#         # voeg de waarden toe aan de return set
#         for iteration, name in enumerate(names_to_merge):
#             pos = np.where(names == name)[0][0]
#             count = m_vals_to_merge[0][iteration]
            
#             vals[0][pos] = vals[0][pos] + count
    
#     merge_vals(names1, vals1)
#     merge_vals(names2, vals2)
    
#     return names, vals

In [ ]:
def get_rows_merged(rows):
    return_rows = np.zeros(shape=(1,len(rows[0])), dtype=int)
        
    # merge de rijen tot een enkele rij
    if len(rows) != 1:

        for row in rows:
            return_rows = return_rows + row

    else:

        return_rows = rows
        
    return return_rows

In [ ]:
# functie testruimte
# contents1 = ['aa bb cc dd']#,
# #             'ee ff gg aa',
# #             'jj kk']
# contents2 = ['aa hh ii jj kk']

# victor = CountVectorizer(ngram_range=(1, 1))
# X1 = victor.fit_transform(contents1)

# een = X1.toarray()
# een_n = victor.get_feature_names_out()

# print(een_n)
# print(een, "\n")

# X2 = victor.fit_transform(contents2)

# twee = X2.toarray()
# twee_n = victor.get_feature_names_out()

# print(twee_n)
# print(twee, "\n")

# # n = np.unique(np.append(een_n, twee_n))
# # v = np.append(twee, een.reshape((len(twee))))
# # print(n, "\n", v, "\n")

# n, v = get_merged_victor(een_n, een, twee_n, twee)
# print(n, "\n", v)

# nmog meer functie testruimte
# df_for_the_test1 = os.path.join("NL", "ParlaMint-NL_2014-04-16-tweedekamer-2.csv")
# df_for_the_test2 = os.path.join("NL", "ParlaMint-NL_2014-04-16-tweedekamer-3.csv")

# victor_output_counts, victor_output_headings = get_victor_from_csv(df_for_the_test1)
# victor_output_counts2, victor_output_headings2 = get_victor_from_csv(df_for_the_test2)
# n, v = get_merged_victor(victor_output_counts, victor_output_headings,
#                          victor_output_counts2, victor_output_headings2)
# print(n, "\n", v)

In [ ]:
# geeft een dict met dates als keys en files als vals
def get_files_per_date(country):
    files = get_csv_files(country)[0][country]
    
    dates_dict = {}
    
    # loop door alle files heen
    for file in files:
        
        date = extract_file_date(file)
        
        if date in dates_dict.keys():
            
            dates_dict[date] += [file]
            
        else:
            
            dates_dict[date] = [file]
            
    return dates_dict

In [ ]:
# maak een csv waar per date alle shingles in staan
def create_shingle_csv(file, headings, counts):
    os.chdir(ngram_csv_dir)
    
    with open(file, 'w', newline='', encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(headings)
        
        if len(counts) == 1:
        
            w.writerow(counts[0])
            
        else:
            
            w.writerow(counts)

In [ ]:
def get_ngram_exists(country, date):
    os.chdir(ngram_csv_dir)
    
    path = os.path.join(country, (date + ".csv"))
    
    return os.path.isfile(path)

In [ ]:
# maakt een csv file aan per datum met shingles
def create_shingle_per_day(country):
    os.chdir(translated_csv_dir)
    
    dates_dict = get_files_per_date(country)
    
    # dingen voor overzicht in prints
    timer = time.time()
    total_dates = len(dates_dict.keys())
    processed_dates = 0
    
    # ga elke date langs
    for date, files in dates_dict.items():
        processed_dates += 1
        dates_percentage = round((processed_dates / total_dates) * 100, 2)
        
        if not get_ngram_exists(country, date):
        
            # ga elke file langs per date
            for iteration, file in enumerate(files):

                path = os.path.join(country, file)

                # basis lijst neerzetten bij eerste iteratie
                if iteration == 0:

                    ngrams, counts = get_victor_from_csv(path)
                    counts = get_rows_merged(counts)[0]
                
                else:

                    new_ngrams, new_counts = get_victor_from_csv(path)

                    # loop de ngram en counts
                    for ngram, count in zip(new_ngrams, get_rows_merged(new_counts)[0]):

                        # verander het ngram nummer als die al bestaat
                        if ngram in ngrams:

                            position = counts[np.where(ngrams == ngram)[0][0]]
                            position = position + count

                        # voeg de ngram toe aan ngrams en counts als die er nog niet is
                        else:

                            ngrams = np.append(ngrams, ngram)
                            counts = np.append(counts, count)
            
            create_shingle_csv(os.path.join(country, (date + ".csv")), ngrams, counts)
            print(f"Progress: {round((time.time() - timer) / 60, 2)}m, Uploaded: {date} ({dates_percentage}%)")
            
        else:

            print(f"Skipping: {date} ({dates_percentage}%)")

In [ ]:
# functie testruimte
real_input = "NL"

tngrams, tcounts = create_shingle_per_day(real_input)

In [ ]:
# # functie testruimte
# ct_test = "NL"

# date_test = get_files_per_date(ct_test)["2014-04-16"]

# for e, d in enumerate(date_test):
#     path = os.path.join(ct_test, d)
    
#     if e == 0:
        
#         liness, countss = get_victor_from_csv(path)
#         countss = get_rows_merged(countss)
        
#     else:
        
#         new_liness, new_countss = get_victor_from_csv(path)
#         plsw = 0
#         for headerr, counttt in zip(new_liness, get_rows_merged(new_countss)[0]):
            
#             if headerr in liness:
                
#                 actual_position = countss[np.where(liness == headerr)[0][0]]
#                 actual_position = actual_position + counttt
                
#             else:
                
#                 liness = np.append(liness, headerr)
#                 countss = np.append(countss, counttt)
    
#     print(len(liness))
        
# create_shingle_csv(path, liness, countss[0])
    
    

In [ ]:
translated_countries = ["BG", "CZ", "DK", "NL", "SI"]

### Oude code met Elastic

In [ ]:
# # Elastic host
# es = Elasticsearch(
#     hosts=[
#             "https://localhost:9200"
#     ],
#     http_auth=("elastic", "NES9DZ-QwhanXAQf9caV"), #basic_auth werkt niet met dsl queries
# #     use_ssl=True,
# #     verify_certs=False,
#     ca_certs="./ca.crt"
# )

In [ ]:
# # translated countries
# translated_countries = ["BG", "CZ", "DK", "NL", "SI"]

# # word counts df
# word_count_csv = 'C:/Users/Asher/Documents/School/_Scriptie/Data/xml_word_counts.csv'

# df_word_count = pd.read_csv(word_count_csv)

# # display(df_word_count)

In [ ]:
# # NOTE: input is {search_key: value, search_key2: value2}
# # query function
# def query(search_dict):
#     processed_search_list = []
    
#     # loop door alle search elements heen
#     for k, v in search_dict.items():
#         processed_search_list.append({"match_phrase" : {k : v}})
        
#     # stel de uitkomst samen
#     result = es.search(
#     index = "search",
#     size = 10000, # TODO: Zorg dat er een groter limit is dan 10000
#     query = {
#         "bool" : {
#             "must": processed_search_list,
#         },
#     })
    
#     return result

In [ ]:
# # https://copyprogramming.com/howto/using-shingles-and-fuzziness-in-elasticsearch-python-dsl
# def make_dsl_query(fields):
#     """
#     Construct a query
#     """
#     my_query = Search(using=es, index="search")
#     if fields['country'] and fields['content_translated']:
#         my_query = my_query.query(Q('bool', should=
#                    [Q("match", name__shingles=fields['country']),
#                     Q("match", surname__shingles=fields['content_translated'])]))
#     return my_query

# # if __name__ == '__main__':
# my_query = make_dsl_query(fields={"country": "NL", "content_translated": "bill"})
# response = my_query.execute()
# print(response)
# for hit in response:
#     print(hit.meta.score, hit.name, hit.surname)
#     break

In [ ]:
# # NOTE: input is {search_key: value, search_key2: value2}
# # query function
# def query_new(search_dict):
#     processed_search_list = []
    
# #     search_dict["analyser"] = "standard_shingle"
    
#     # loop door alle search elements heen
#     for k, v in search_dict.items():
#         processed_search_list.append({"match_phrase" : {k : v}})
        
#     # stel de uitkomst samen
#     result = es.search(
#     index = "search",
#     size = 10000, # TODO: Zorg dat er een groter limit is dan 10000
#     query = {
#         "bool" : {
#             "must": processed_search_list,
#         },
#     })
    
#     return result

# result = query_new({"country": "NL", "content_translated": "bill"})

# print(result["hits"]["hits"])

### Query testing

In [ ]:
# # TODO: debate uploads opniew runnen om andere vertaalde landen ook toe te voegen

# countries_returned = {}

# for ctr in translated_countries:
#     query_dit = query({"content_translated":"bill", "country":ctr})
    
#     for line in query_dit["hits"]["hits"]:

#         ct = line["_source"]["country"]

#         if ct in countries_returned.keys():

#             countries_returned[ct] += 1

#         else:

#             countries_returned[ct] = 1
    
# print(countries_returned)